<a href="https://colab.research.google.com/github/agderegibus/MarketAnalysis/blob/master/MonitorDeON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import pandas as pd


In [122]:
with open("CUENTAS.xlsx",mode="rb") as excel_file:
    CUENTAS = pd.read_excel("CUENTAS.xlsx")
cuentas = CUENTAS.drop(labels=['CUIT/CUIL/CDI',"Persona", 'MC'], axis=1)
print(cuentas)

     CODIGOCUENTA  Persona ID  MC Cód. Tipo Cuenta
0            1001           2      320          AN
1            1004        2752        4    Terceros
2            1005        4873        5    Terceros
3           10062        2257      324      Propia
4            1008          10        8    Terceros
..            ...         ...      ...         ...
746         98100       20629       51      Propia
747          9822        6425      128      Propia
748          9861        6458      299      Propia
749          9954        4088      133      Propia
750          9980        1932      131      Propia

[751 rows x 4 columns]


In [123]:
219    26.0         1026         1026.0   
3331  415.0         1415       158170.0   
3559  327.0         1327        15935.0   
4668  345.0         1345        92795.0   
4765  345.0         1345       100244.0   
4823  345.0         1345        78391.0   
5249  356.0         1356         1356.0   
5261  356.0        35719        35719.0   
5762  334.0        54500        29192.0   
5831  378.0         1378         1378.0   
5844  379.0         1379       158654.0   
7832  175.0       148000       148000.0 

SyntaxError: ignored

In [124]:
SALDOS = pd.read_excel("SALDITOS.xlsx", 'Detalle')
#inicial = (SE[SE["Estado"] == "Inicial"]).sort_values("MC Código")


In [125]:
# CONSEGUIR SOLO LAS QUE TIENEN SOLICITUD DE EXTRACCIO

EXTRAC = SALDOS[SALDOS['Egreso No Verificado'] != 0]
df = EXTRAC.astype({"Cuenta Liquidación Código": int}, errors='raise')
df.rename(columns = {'Cuenta Liquidación Código':'CODIGOCUENTA'}, inplace = True)

print(df)


       ALyC  CODIGOCUENTA  Cuenta Código  \
471   320.0          1140         3326.0   
756   218.0          1218       157951.0   
2037  313.0          1313        89071.0   
2484  405.0          1405        98120.0   
2486  405.0          1405       101663.0   
2779  405.0          1405        60583.0   
2781  405.0          1405        41396.0   
2784  405.0          1405       142175.0   
2833  405.0          1405        34584.0   
2894  405.0          1405        92553.0   
2901  405.0          1405        98119.0   
3005  405.0          1405        59738.0   
3652  419.0          1419        72615.0   
3776  323.0          1323       100967.0   
4342  329.0          1329       160390.0   
4351  329.0          1329        72372.0   
4419  325.0         13875        13875.0   
4568  334.0          1334        29404.0   
5128  345.0          1345        85614.0   
5161  345.0          1345       148594.0   
5177  345.0          1345       155528.0   
5217  345.0          1345       

In [126]:
df = df.merge(cuentas,on='CODIGOCUENTA',how="left")
print(df)

     ALyC  CODIGOCUENTA  Cuenta Código  \
0   320.0          1140         3326.0   
1   218.0          1218       157951.0   
2   313.0          1313        89071.0   
3   405.0          1405        98120.0   
4   405.0          1405       101663.0   
5   405.0          1405        60583.0   
6   405.0          1405        41396.0   
7   405.0          1405       142175.0   
8   405.0          1405        34584.0   
9   405.0          1405        92553.0   
10  405.0          1405        98119.0   
11  405.0          1405        59738.0   
12  419.0          1419        72615.0   
13  323.0          1323       100967.0   
14  329.0          1329       160390.0   
15  329.0          1329        72372.0   
16  325.0         13875        13875.0   
17  334.0          1334        29404.0   
18  345.0          1345        85614.0   
19  345.0          1345       148594.0   
20  345.0          1345       155528.0   
21  345.0          1345        91816.0   
22  345.0          1345        788

In [127]:
df["TIENE SALDO?"] = (df["Saldo Inicial"] + df["Egreso No Verificado"] + df["Requerido Del Día"]) 


In [ ]:
def highlight_greaterthan(s, threshold, column):
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = s.loc[column] >= threshold
    return ['background-color: green' if is_max.any() else '' for v in is_max]


df.style.apply(highlight_greaterthan, threshold=1.0, column=['TIENE SALDO?'], axis=1)

def sinsaldo(s, threshold, column):
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = s.loc[column] < threshold
    return ['background-color: red' if is_max.any() else '' for v in is_max]

df.style.apply(highlight_greaterthan, threshold=1.0, column=['TIENE SALDO?'], axis=1)


In [132]:
def highlight_rows(row):
    value = row.loc['TIENE SALDO?']
    if value < 0:
        color = '#FF0000' # Red
    elif value > 0:
        color = '#027d0d' # Green
    else:
        color = '#bfc213' # Blue
    return ['background-color: {}'.format(color) for r in row]

df.style.apply(highlight_rows, axis=1)

,ALyC,CODIGOCUENTA,Cuenta Código,Cuenta,Finalidad,Moneda,Saldo Inicial,Movimientos Verificados,Ingreso No Verificado,Egreso No Verificado,Saldo Final Verificado,Saldo Final No Verificado,Requerido Del Día,Saldo Consolidado Verificado,Saldo Consolidado No Verificado,Persona ID,MC Cód.,Tipo Cuenta,TIENE SALDO?
0,320.000000,1140,3326.000000,COMSA S.A.,Márgenes,Pesos,8226406.920000,0.000000,0.000000,-310.040000,8226406.920000,8226096.880000,0.000000,8226406.920000,8226096.880000,6674,320,AN,8226096.880000
1,218.000000,1218,157951.000000,EMPRESA DUVAL FLORES SOCIEDAD ANONIMA,Márgenes,Pesos,1627549.517832,0.000000,0.000000,-862368.000000,1627549.517832,765181.517832,28965.222168,1656514.740000,794146.740000,2309,218,Terceros,794146.740000
2,313.000000,1313,89071.000000,PACHECO CHAPAS SA,Márgenes,Pesos,27313594.270000,52448.783367,0.000000,-8371369.760000,27366043.053367,18994673.293367,-10700000.000000,16666043.053367,8294673.293367,8491,313,Terceros,8242224.510000
3,405.000000,1405,98120.000000,PERRI LILIANA BEATRIZ,Márgenes,Pesos,903991.680000,0.000000,0.000000,-906553.650000,903991.680000,-2561.970000,0.000000,903991.680000,-2561.970000,6695,405,Terceros,-2561.970000
4,405.000000,1405,101663.000000,CAMPOS JOSE LUIS,Márgenes,Pesos,1100045.950000,0.000000,0.000000,-1093120.000000,1100045.950000,6925.950000,0.000000,1100045.950000,6925.950000,6695,405,Terceros,6925.950000
5,405.000000,1405,60583.000000,AMADO JORGE LUIS,Márgenes,Pesos,1434127.060000,0.000000,0.000000,-1430664.400000,1434127.060000,3462.660000,0.000000,1434127.060000,3462.660000,6695,405,Terceros,3462.660000
6,405.000000,1405,41396.000000,VAZQUEZ ALBERTINA,Márgenes,Pesos,786107.980000,-772044.980000,0.000000,-14063.000000,14063.000000,0.000000,0.000000,14063.000000,0.000000,6695,405,Terceros,772044.980000
7,405.000000,1405,142175.000000,CRUZ LUCIANA SOLEDAD,Márgenes,Pesos,3498.880000,0.000000,0.000000,-3264.630000,3498.880000,234.250000,0.000000,3498.880000,234.250000,6695,405,Terceros,234.250000
8,405.000000,1405,34584.000000,RAMIREZ FERNANDO GABRIEL,Márgenes,Pesos,651328.680000,0.000000,0.000000,-649861.720000,651328.680000,1466.960000,0.000000,651328.680000,1466.960000,6695,405,Terceros,1466.960000
9,405.000000,1405,92553.000000,VILLARES DANIEL ALBERTO,Márgenes,Pesos,633798.180000,0.000000,0.000000,-631428.300000,633798.180000,2369.880000,0.000000,633798.180000,2369.880000,6695,405,Terceros,2369.880000
